In [1]:
import twelvedata
f = open("keys.txt", "r")
key = f.read()
my_key = key
# from keys import twelveDataKey as api_key
from twelvedata import TDClient

### 1. Getting the Data

In [14]:
td = TDClient(apikey = my_key)

# sym is the symbol (AAPL, MSFT, IBM); start is the start date. format: 2020-01-01; end is the end date.
def get_data(sym, start, end):
    ts = td.time_series(
         symbol=sym,
         interval="15min" ,
         start_date=start, 
         end_date=end,
         type = "Common Stock",
         outputsize = 5000

    ).as_pandas()
    ts.to_csv('stock data/'+sym+'_'+end+'.csv',encoding='utf-8')
    
get_data('AAPL', '2020-10-31','2024-10-31')
get_data('AAPL', '2020-10-31','2024-01-26')
get_data('AAPL', '2020-10-31','2023-04-20')
get_data('AAPL', '2020-10-31','2022-07-14')
get_data('AAPL', '2020-10-31','2021-10-05')
get_data('AAPL', '2020-10-31','2020-12-29')
get_data('AAPL', '2020-01-01','2020-10-31')
get_data('AAPL', '2020-01-01','2020-01-29')

get_data('MSFT', '2020-10-31','2024-10-31')
get_data('MSFT', '2020-10-31','2024-01-26')
get_data('MSFT', '2020-10-31','2023-04-20')
get_data('MSFT', '2020-10-31','2022-07-14')
get_data('MSFT', '2020-10-31','2021-10-05')
get_data('MSFT', '2020-10-31','2020-12-29')
get_data('MSFT', '2020-01-01','2020-10-31')
get_data('MSFT', '2020-01-01','2020-01-29')

get_data('IBM', '2020-10-31','2024-10-31')
get_data('IBM', '2020-10-31','2024-01-26')
get_data('IBM', '2020-10-31','2023-04-20')
get_data('IBM', '2020-10-31','2022-07-14')
get_data('IBM', '2020-10-31','2021-10-05')
get_data('IBM', '2020-10-31','2020-12-29')
get_data('IBM', '2020-01-01','2020-10-31')
get_data('IBM', '2020-01-01','2020-01-29')

### 2. Concatenate all the files

In [25]:
import glob
import os
import pandas as pd

# path = r'stock data' # use your path
all_files = glob.glob('stock data/*.csv')
li = []

for filename in all_files:
    df = pd.read_csv(filename)
    if 'IBM' in filename:
        df['Symbol'] = 'IBM'
    elif 'AAPL' in filename:
        df['Symbol'] = 'AAPL'
    elif 'MSFT' in filename:
        df['Symbol'] = 'MSFT'
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.to_csv('four_years.csv',encoding='utf-8')

['stock data/AAPL_2024-01-26.csv', 'stock data/MSFT_2020-01-29.csv', 'stock data/AAPL_2020-10-31.csv', 'stock data/IBM_2024-01-26.csv', 'stock data/IBM_2020-10-31.csv', 'stock data/AAPL_2020-01-29.csv', 'stock data/MSFT_2024-01-26.csv', 'stock data/MSFT_2020-10-31.csv', 'stock data/IBM_2020-01-29.csv', 'stock data/MSFT_2020-12-29.csv', 'stock data/AAPL_2022-07-14.csv', 'stock data/IBM_2023-04-20.csv', 'stock data/IBM_2024-10-31.csv', 'stock data/AAPL_2021-10-05.csv', 'stock data/IBM_2022-07-14.csv', 'stock data/AAPL_2023-04-20.csv', 'stock data/AAPL_2024-10-31.csv', 'stock data/IBM_2021-10-05.csv', 'stock data/MSFT_2023-04-19.csv', 'stock data/MSFT_2022-07-14.csv', 'stock data/AAPL_2020-12-29.csv', 'stock data/MSFT_2021-10-05.csv', 'stock data/IBM_2020-12-29.csv', 'stock data/MSFT_2024-10-31.csv']


### 3. Make the hash function

In [26]:
def df_hexdigest(saved_df):
    '''
        saved_df should be a dataframe with columns ['datetime', 'open', 'high', 'low', 'close', 'volume', 'Symbol']
        and with all data points between 2020-01-01 and 2024-06-30 (both dates inclusive), for  AAPL, MSFT and IBM.
    '''

    # Convert 'datetime' column to datetime objects if it's not already
    import pandas as pd

    saved_df['datetime'] = pd.to_datetime(saved_df['datetime'])

    # Sort the DataFrame by the 'datetime' column
    saved_df = saved_df.sort_values('datetime')

    # Display the sorted DataFrame (optional)
    # print(saved_df)
    saved_df = saved_df.reset_index(drop=True)
    saved_df

    df_filtered = saved_df[(saved_df['datetime'] >= '2020-01-01') & (saved_df['datetime'] <= '2024-06-30')]
    df_filtered

    # Sort the DataFrame by the 'datetime' column
    df_filtered_sorted = df_filtered.sort_values('datetime')

    df_filtered_sorted = df_filtered_sorted.reset_index(drop=True)
    df_filtered_sorted.head()

    df_filtered_sorted_deduped = df_filtered_sorted.drop_duplicates(subset=['datetime', 'open', 'high', 'low', 'close', 'volume', 'Symbol'], keep='first')

    printed_string = df_filtered_sorted_deduped[['datetime', 'Symbol']].to_string(index=False, header=False)
    # print(printed_string[:2000])
    import hashlib
    md5_hash = hashlib.md5(printed_string.encode()) # Use hashlib.md5() to create the hash object
    return md5_hash.hexdigest()

df = pd.read_csv('four_years.csv')
fourYearHash = df_hexdigest(df)

### References

1. https://support.twelvedata.com/en/collections/3198976-how-to
2. https://twelvedata.com/blog/get-high-quality-financial-data-directly-into-python
3. https://www.youtube.com/watch?v=LMSYFJVcEo8